## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-08-08-56-00 +0000,wsj,Euro Rises After ECB’s Schnabel Signals Next M...,https://www.wsj.com/finance/commodities-future...
1,2025-12-08-08-52-17 +0000,nyt,Thailand Launches Airstrikes on Cambodia in Ne...,https://www.nytimes.com/2025/12/08/world/asia/...
2,2025-12-08-08-51-59 +0000,nypost,Marjorie Taylor Greene spars with ‘60 Minutes’...,https://nypost.com/2025/12/08/us-news/marjorie...
3,2025-12-08-08-32-30 +0000,wsj,Stock Market Today: Nasdaq Futures Lead Gains;...,https://www.wsj.com/livecoverage/stock-market-...
4,2025-12-08-08-27-57 +0000,bbc,One hundred abducted schoolchildren released i...,https://www.bbc.com/news/articles/cvgjvg4ylwpo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
108,trump,29
13,new,17
224,will,9
245,he,9
296,fire,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
56,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...,66
94,2025-12-07-22-30-00 +0000,wsj,President Trump criticized as disloyal an indi...,https://www.wsj.com/politics/elections/trump-l...,61
133,2025-12-07-19-42-14 +0000,cbc,Trump's new security strategy aligns with Russ...,https://www.cbc.ca/news/world/russia-trump-nat...,59
169,2025-12-07-13-42-20 +0000,cbc,"Israel's Netanyahu says he will meet Trump, se...",https://www.cbc.ca/news/world/netanyahu-says-h...,58
14,2025-12-08-07-31-03 +0000,bbc,Trump says $72bn Netflix-Warner Bros deal 'cou...,https://www.bbc.com/news/articles/cn815egjqjpo...,53


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
56,66,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...
179,36,2025-12-07-12-33-07 +0000,nypost,Hochul will drive away New Yorkers if she supp...,https://nypost.com/2025/12/07/us-news/pataki-h...
62,32,2025-12-08-01-54-25 +0000,nypost,JPMorgan Chase’s Jamie Dimon claims AI will no...,https://nypost.com/2025/12/07/business/jamie-d...
102,28,2025-12-07-21-44-45 +0000,cbc,Conservative motion will force Liberals to 'pu...,https://www.cbc.ca/news/politics/poilievre-con...
101,26,2025-12-07-21-46-24 +0000,nypost,Holiday parade in California’s wealthy ‘Americ...,https://nypost.com/2025/12/07/us-news/holiday-...
58,24,2025-12-08-02-19-24 +0000,bbc,Former Australian deputy PM joins anti-immigra...,https://www.bbc.com/news/articles/cqxq8g4d873o...
39,24,2025-12-08-04-15-17 +0000,nyt,China’s Trade Surplus Climbs Past $1 Trillion ...,https://www.nytimes.com/2025/12/07/business/ch...
135,24,2025-12-07-19-29-38 +0000,nypost,Water leak at the Louvre damages hundreds of r...,https://nypost.com/2025/12/07/world-news/water...
152,24,2025-12-07-17-40-13 +0000,bbc,Nightclub fire in India's Goa kills 25 staff a...,https://www.bbc.com/news/articles/cdj82d0nd4eo...
19,23,2025-12-08-07-01-00 +0000,wsj,"German Shipbuilder TKMS Reports Rise in Sales,...",https://www.wsj.com/business/earnings/german-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
